In [1]:
import os
import sys

from pathlib import Path

sys.path = []
sys.path.append(os.getcwd())

env_packages = '.env/lib/python3.7/site-packages/'

sys.path.append(Path(os.getcwd(),env_packages).as_posix())
sys.path.append('/home/saraiva/miniconda3/lib/python37.zip')
sys.path.append('/home/saraiva/miniconda3/lib/python3.7')
sys.path.append('/home/saraiva/miniconda3/lib/python3.7/lib-dynload')

In [2]:
from transformers import pipeline

#Carrega classifier padrao
classifier = pipeline('sentiment-analysis')
print(classifier.model.name_or_path)

#Realiza escoragem
results = classifier(["We are very happy to show you the 🤗 Transformers library.",
           "We hope you don't hate it.",
           'Eu adoro esse produto!!'])

for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

/home/saraiva/Documentos/GitHub/knowledge-acervus/fine-tuning-bert/.env/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


distilbert-base-uncased-finetuned-sst-2-english
label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309
label: POSITIVE, with score: 0.9968


In [3]:
#Carregando um outr classifier de 
classifier = pipeline('sentiment-analysis',
                      model="nlptown/bert-base-multilingual-uncased-sentiment")
print(classifier.model.name_or_path)
      
results = classifier(["We are very happy to show you the 🤗 Transformers library.",
           "We hope you don't hate it.",
           'Eu adoro esse produto!!'])

for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

nlptown/bert-base-multilingual-uncased-sentiment
label: 5 stars, with score: 0.7725
label: 5 stars, with score: 0.2365
label: 5 stars, with score: 0.7192


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

tokenizer(["I wouldn't like this potatoe",
           "We really should love transformers, it's so simple to play with it",
           "Ah não tem problema!"],padding=True)

{'input_ids': [[101, 151, 11008, 10115, 112, 162, 11531, 10372, 14381, 12311, 10111, 102, 0, 0, 0, 0, 0], [101, 11312, 25165, 14693, 11157, 58263, 117, 10197, 112, 161, 10297, 15511, 10114, 11923, 10171, 10197, 102], [101, 25577, 11373, 12492, 17483, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [163]:
if pt_model.name_or_path != "distilbert-base-uncased-finetuned-sst-2-english" :
    #instancia novo modelo
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs = tokenizer(["We are very happy to show you the 🤗 Transformers library.",
                    "Whatever dude!",
                    "Merry Christmas! Love in your life bro."],
                   padding=True,
                   return_token_type_ids=False,
                   max_length=512,
                   truncation=True,
                   return_tensors='pt') #precisamos retornar tensores para inputar no modelo//

pt_outputs = pt_model(**inputs)

In [249]:
import torch.nn.functional as F
pt_predictions = F.softmax(pt_outputs[0], dim=-1)

#Ultima camada de ativacao antes de softmax
print(pt_predictions)

#Obtendo predicoes por linha
print(pt_predictions.argmax(dim=1))

tensor([[2.2043e-04, 9.9978e-01],
        [9.4621e-01, 5.3790e-02],
        [1.2783e-04, 9.9987e-01]], grad_fn=<SoftmaxBackward>)
tensor([1, 0, 1])


### DistilBert Model

In [246]:
from transformers import AutoModelForMaskedLM
from transformers import FillMaskPipeline


if pt_model.name_or_path != "distilbert-base-uncased" :
    #instancia novo modelo
    model_name = "distilbert-base-uncased"
    pt_model = AutoModelForMaskedLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)  
    
fmp = FillMaskPipeline(model = pt_model,
                       tokenizer = tokenizer)

In [284]:
fmp(['The capital of Italy is [MASK]!'])

[{'sequence': '[CLS] the capital of italy is naples! [SEP]',
  'score': 0.1677890568971634,
  'token': 10553,
  'token_str': 'naples'},
 {'sequence': '[CLS] the capital of italy is milan! [SEP]',
  'score': 0.13662037253379822,
  'token': 6954,
  'token_str': 'milan'},
 {'sequence': '[CLS] the capital of italy is rome! [SEP]',
  'score': 0.13033248484134674,
  'token': 4199,
  'token_str': 'rome'},
 {'sequence': '[CLS] the capital of italy is florence! [SEP]',
  'score': 0.09330018609762192,
  'token': 7701,
  'token_str': 'florence'},
 {'sequence': '[CLS] the capital of italy is turin! [SEP]',
  'score': 0.08116588741540909,
  'token': 13667,
  'token_str': 'turin'}]

In [362]:
from transformers import FeatureExtractionPipeline
from transformers import AutoModel

def cosine_sim(a,b):
    return a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))

pt_model_fext = AutoModel.from_pretrained("distilbert-base-uncased")
feat_ext = FeatureExtractionPipeline(model = pt_model_fext,
                          tokenizer = tokenizer)

# sent_a = 'The girl next to the exit is really beautiful, her hair shines through the light'
# sent_b = 'That woman in the entrance is gorgeous' 

sent_a = 'That dog is really cute'
sent_b = 'I want to play soccer with my friends' 

result = np.array(feat_ext([sent_a,
                            sent_b]))

In [363]:
cosine_sim(result[0].mean(axis=0),result[1].mean(axis=0))

0.7375967992184406

In [364]:
cosine_sim(result[0].max(axis=0),result[1].max(axis=0))

0.8809934666196906

### PyTorch Learning - Autograd

In [103]:
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [104]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [105]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [106]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [110]:
x=torch.tensor(3.,requires_grad=True)
w=torch.tensor(2.,requires_grad=True)
b=torch.tensor(4.,requires_grad=True)

y=w*x+b
print(y)

y.backward()
print(x.grad)
print(w.grad)
print(b.grad)

tensor(10., grad_fn=<AddBackward0>)
tensor(2.)
tensor(3.)
tensor(1.)


In [265]:
0.42857142857142855/(1+0.42857142857142855)

0.3